In [1]:
import requests
from bs4 import BeautifulSoup

def get_articles_from_first_page():
    base_url = "https://www.gukbangnews.com"
    first_page_url = f"{base_url}/news/articleList.html?page=1&sc_sub_section_code=S2N5"

    res = requests.get(first_page_url)
    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.text, "html.parser")

    article_data = []

    # 기사 제목 추출
    title_tags = soup.select("h4.titles > a")
    for tag in title_tags:
        title = tag.text.strip()
        href = tag['href']
        article_url = base_url + href

        # 기사 내용 가져오기
        article_res = requests.get(article_url)
        article_res.encoding = 'utf-8'
        article_soup = BeautifulSoup(article_res.text, "html.parser")

        content_area = article_soup.find("article", {"id": "article-view-content-div"})
        if content_area:
            paragraphs = content_area.find_all("p")
            content_text = ' '.join([p.get_text(strip=True) for p in paragraphs])

            article_data.append({
                "기사제목": title,
                "기사내용": content_text
            })

    return article_data


In [4]:
data = get_articles_from_first_page()
for d in data:
    print(f"[{d['기사제목']}]\n{d['기사내용']}...\n")

[KB국민은행-KAI, 1조원 규모 상생협력 업무협약]
[국방신문=박세정 전문기자] 한국항공우주산업(KAI)와 KB국민은행이 항공우주분야 경쟁력 강화를 통한 중소 협력업체와 동반성장 및 및 세계시장 진출을 위해 손을 잡았다. KB국민은행과 KAI는 11일 서울시 영등포구 KB국민은행 신관에서 협약식을 갖고 ‘상생협력 및 공동발전’을 위한 업무협약(MOU)을 체결했다. 강구영 KAI 사장과 이환주 KB국민은행장은 이날 협약을 국가 주력사업인 항공우주 분야의 경쟁력 강화와 협력업체 동반성장, 세계시장 진출 확대하기 위해 양 기관의 상호 협력체계를 구축하기로 했다. 이번 협약에 따라 KB국민은행이 KAI에 지원하는 금융지원 규모는 최대 1조원이다. KB국민은행은 ▲미래사업 투자지원 ▲수출금융 ▲수출입 및 시설투자 관련 운영자금 대출 등 KAI에서 진행하는 사업에 대한 금융 솔루션을 제공할 예정이다. 또 KAI 협력업체를 대상으로 한 동반성장 금융지원을 통해 중소기업과 상생금융을 강화한다. 강구영 KAI 사장은 “항공우주산업은 높은 기술력과 장기적인 투자가 필수적인 만큼 안정적인 금융지원이 매우 중요하다”며 “이번 협약을 통해 미래사업 투자와 글로벌시장 확대를 위한 기반이 더욱 강화될 것으로 기대한다”고 밝혔다. 이환주 KB국민은행장은 “대한민국의 항공우주 및 방위산업을 선도해 온 KAI와 협약을 체결하게 되어 뜻깊게 생각한다”며 “이번 협약이 단순한 금융지원을 넘어 대한민국 항공우주 산업의 경쟁력을 높이고 지속가능한 성장을 함께 이루어가는 초석이 되길 바란다”고 말했다....

[방산 AI 스타트업 퀀텀에어로 37억원 Seed 투자 유치]
[국방신문=양기반 기자] 방위산업 인공지능(AI) 스타트업 퀀텀에어로가 ST캐피탈과 오픈워터엔젤스, 전략적 투자자(SI) 2곳으로부터 Seed 투자 37억원을 유치해 총 누적 투자금 약 47억원을 달성했다고 18일 밝혔다. 이번 투자 유치로 퀀텀에어로는 방산 AI 기술 경쟁력을 강화하는 동시에 대한민국의 미래 전장 환경 혁신 및 

In [9]:
import time
import csv
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from urllib.parse import urljoin

# 셀레니움 설정 (headless 옵션 주석처리하여 창이 뜨게 함)
options = Options()
# options.add_argument("--headless")  # 디버깅 시에는 주석 처리
driver = webdriver.Chrome(options=options)

def get_articles_from_page(page_url):
    driver.get(page_url)
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, "html.parser")

    articles_data = []
    article_blocks = soup.select("h4.titles > a")

    for article in article_blocks:
        title = article.text.strip()
        link = urljoin(page_url, article['href'])

        driver.get(link)
        time.sleep(1)
        detail_soup = BeautifulSoup(driver.page_source, "html.parser")
        content_div = detail_soup.select_one("article#article-view-content-div")

        if content_div:
            paragraphs = content_div.find_all("p")
            content = "\n".join([p.text.strip() for p in paragraphs])
        else:
            content = ""

        articles_data.append({
            "기사제목": title,
            "기사내용": content
        })

    return articles_data

def crawl_all_articles(base_url, output_csv):
    all_articles = []

    for page in range(1, 21):  # 1부터 20페이지까지
        page_url = f"{base_url}?page={page}&sc_sub_section_code=S2N5&view_type=sm"
        print(f"크롤링 중: {page} 페이지")
        articles = get_articles_from_page(page_url)

        if not articles:
            print(f"{page} 페이지에 기사 없음. 종료합니다.")
            break

        all_articles.extend(articles)

    with open(output_csv, mode="w", newline="", encoding="utf-8-sig") as f:
        writer = csv.DictWriter(f, fieldnames=["기사제목", "기사내용"])
        writer.writeheader()
        writer.writerows(all_articles)

    print(f"총 {len(all_articles)}개의 기사를 저장했습니다.")

# 사용 예시
base_url = "https://www.gukbangnews.com/news/articleList.html"
output_csv = "국방신문_방위산업.csv"
crawl_all_articles(base_url, output_csv)

# 사용 후 드라이버 종료
driver.quit()


크롤링 중: 1 페이지
크롤링 중: 2 페이지
크롤링 중: 3 페이지
크롤링 중: 4 페이지
크롤링 중: 5 페이지
크롤링 중: 6 페이지
크롤링 중: 7 페이지
크롤링 중: 8 페이지
크롤링 중: 9 페이지
크롤링 중: 10 페이지
크롤링 중: 11 페이지
크롤링 중: 12 페이지
크롤링 중: 13 페이지
크롤링 중: 14 페이지
크롤링 중: 15 페이지
크롤링 중: 16 페이지
크롤링 중: 17 페이지
크롤링 중: 18 페이지
크롤링 중: 19 페이지
크롤링 중: 20 페이지
총 416개의 기사를 저장했습니다.
